In [187]:
import pandas as pd
import json
import urllib
from urllib.request import urlopen
import re
import math

In [122]:
def refactor_values(data):
    new_data = {}
    for key, value in data.items():
            if value != '':
                new_data[key] = value
            if isinstance(value, str) and '<' in value:
                new_value = value.split('<')[1]
                new_data[key] = int(new_value)
    return new_data

In [294]:
url = "https://arsoxmlwrapper.app.grega.xyz/api/air/archive"

response = urlopen(url)
data_json = json.loads(response.read())

json_object = json.dumps(data_json)


In [295]:
save_file = open("data/raw/air", "w")  
json.dump(data_json, save_file)  
save_file.close()  

In [296]:
save_file = open("../data/raw/air", "w")  
json.dump(data_json, save_file)  
save_file.close()  

In [297]:
load_file = open('data/raw/air')
dataJson = json.load(load_file)
load_file.close()

In [298]:
df = pd.DataFrame()

for i in range(len(dataJson)):
    data = dataJson[i]
    dictData = json.loads(data['json'])
    
    df1 = pd.json_normalize(dictData['arsopodatki']['postaja'])
    df = pd.concat([df, df1])#, axis=1, join="inner")

In [299]:
df = df.reset_index(drop=True)

In [300]:
df.dtypes

pm2.5             object
nadm_visina        int64
datum_od          object
merilno_mesto     object
o3                object
benzen            object
ge_sirina        float64
pm10              object
co                object
no2               object
sifra             object
datum_do          object
ge_dolzina       float64
so2               object
dtype: object

In [302]:
df.head()

,pm2.5,nadm_visina,datum_od,merilno_mesto,o3,benzen,ge_sirina,pm10,co,no2,sifra,datum_do,ge_dolzina,so2
0,34.0,299,2023-02-22 21:00,LJ Bežigrad,61,,46.065851,39.0,0.3,19,E403,2023-02-22 22:00,14.517454,NaN
1,38.0,305,2023-02-22 21:00,LJ Celovška,NaN,NaN,46.072399,46.0,NaN,38,E405,2023-02-22 22:00,14.491849,NaN
2,35.0,293,2023-02-22 21:00,LJ Vič,NaN,NaN,46.037791,40.0,NaN,NaN,E404,2023-02-22 22:00,14.494001,NaN
3,88.0,388,2023-02-22 21:00,Kranj,NaN,NaN,46.242115,95.0,NaN,NaN,E417,2023-02-22 22:00,14.366963,NaN
4,33.0,270,2023-02-22 21:00,MB Titova,NaN,2.3,46.559202,41.0,NaN,32,E407,2023-02-22 22:00,15.656191,NaN


In [246]:
####

In [247]:
#file = open('../data/raw/data.json', 'r', encoding='utf-8')
#raw = json.load(file)
#file.close()



#df = pd.DataFrame()

#for i in range(len(raw)):
#    jdata = json.loads(raw[i]['json'])
#    station = jdata['arsopodatki']['postaja']
#    for i in range(len(station)):
#        data = station[i]
#        data = refactor_values(data)
#        df = pd.concat([df, pd.json_normalize(data)])  
        
#df

In [248]:
####

In [304]:
print(df.isnull().sum())

pm2.5             14480
nadm_visina           0
datum_od              0
merilno_mesto         0
o3                72400
benzen           130320
ge_sirina          7240
pm10              14480
co               144800
no2               72400
sifra                 0
datum_do              0
ge_dolzina         7240
so2              130320
dtype: int64


In [305]:
cat_cols = df.select_dtypes(include=['object']).columns
print(cat_cols)
num_cols = df.select_dtypes(include=['number']).columns
print(num_cols)

Index(['pm2.5', 'datum_od', 'merilno_mesto', 'o3', 'benzen', 'pm10', 'co',
       'no2', 'sifra', 'datum_do', 'so2'],
      dtype='object')
Index(['nadm_visina', 'ge_sirina', 'ge_dolzina'], dtype='object')


In [306]:
for col in num_cols:
    df[col].fillna((df[col].mean()), inplace=True)

df_location = pd.get_dummies(df['merilno_mesto'])
df = pd.concat([df, df_location], axis=1).reindex(df.index)
df.pop('merilno_mesto')
df.pop('sifra')
df.pop('datum_od')
df.pop('datum_do')

df

,pm2.5,nadm_visina,o3,benzen,ge_sirina,pm10,co,no2,ge_dolzina,so2,...,MB Vrbanski,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje
0,34.0,299,61,,46.065851,39.0,0.3,19,14.517454,NaN,...,0,0,0,0,0,0,0,0,0,0
1,38.0,305,NaN,NaN,46.072399,46.0,NaN,38,14.491849,NaN,...,0,0,0,0,0,0,0,0,0,0
2,35.0,293,NaN,NaN,46.037791,40.0,NaN,NaN,14.494001,NaN,...,0,0,0,0,0,0,0,0,0,0
3,88.0,388,NaN,NaN,46.242115,95.0,NaN,NaN,14.366963,NaN,...,0,0,0,0,0,0,0,0,0,0
4,33.0,270,NaN,2.3,46.559202,41.0,NaN,32,15.656191,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152035,26,241,17,NaN,46.131416,30,NaN,30,15.000906,4.0,...,0,0,0,0,0,0,0,0,0,1
152036,14,290,NaN,NaN,46.144213,15,NaN,NaN,15.088097,NaN,...,0,0,0,0,0,0,0,0,0,0
152037,41,220,9,NaN,45.802080,44,NaN,19,15.182060,NaN,...,0,0,0,0,1,0,0,0,0,0
152038,10,540,36,,45.561344,13,NaN,4,14.862816,2.0,...,0,0,0,0,0,0,0,0,0,0


In [307]:
df.dtypes

pm2.5                   object
nadm_visina              int64
o3                      object
benzen                  object
ge_sirina              float64
pm10                    object
co                      object
no2                     object
ge_dolzina             float64
so2                     object
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
dtype: object

In [308]:
stevilski = df[(["pm2.5", "nadm_visina", "o3", "benzen", "ge_sirina", "pm10", "co", "no2", "ge_dolzina", "so2"])]

In [309]:
stevilski = stevilski.apply(pd.to_numeric, args=('coerce',))

In [310]:
stevilski.isnull().sum()

pm2.5           23442
nadm_visina         0
o3              82599
benzen         141997
ge_sirina           0
pm10            22074
co             145387
no2             84334
ge_dolzina          0
so2            133794
dtype: int64

In [311]:
stevilski['pm2.5'].fillna(stevilski['pm2.5'].mean(), inplace=True)
stevilski['nadm_visina'].fillna(stevilski['nadm_visina'].mean(), inplace=True)
stevilski['o3'].fillna(stevilski['o3'].mean(), inplace=True)
stevilski['benzen'].fillna(stevilski['benzen'].mean(), inplace=True)
stevilski['ge_sirina'].fillna(stevilski['ge_sirina'].mean(), inplace=True)
stevilski['pm10'].fillna(stevilski['pm10'].mean(), inplace=True)
stevilski['co'].fillna(stevilski['co'].mean(), inplace=True)
stevilski['no2'].fillna(stevilski['no2'].mean(), inplace=True)
stevilski['ge_dolzina'].fillna(stevilski['ge_dolzina'].mean(), inplace=True)
stevilski['so2'].fillna(stevilski['so2'].mean(), inplace=True)
stevilski.isnull().sum()

pm2.5          0
nadm_visina    0
o3             0
benzen         0
ge_sirina      0
pm10           0
co             0
no2            0
ge_dolzina     0
so2            0
dtype: int64

In [358]:
df_drop = df.drop(columns=stevilski)

In [359]:
df = pd.concat([stevilski, df_drop], axis=1, join="inner")

In [360]:
df.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [361]:
df.to_csv('../data/processed/data_air.csv', index=False)

In [362]:
df.to_csv('data/processed/data_air.csv', index=False)

In [363]:
df.to_csv('../src/data/data/processed/data_air.csv', index=False)

In [364]:
df.columns

Index(['pm2.5', 'nadm_visina', 'o3', 'benzen', 'ge_sirina', 'pm10', 'co',
       'no2', 'ge_dolzina', 'so2', 'CE Ljubljanska', 'CE bolnica', 'Hrastnik',
       'Iskrba', 'Koper', 'Kranj', 'Krvavec', 'LJ Bežigrad', 'LJ Celovška',
       'LJ Vič', 'MB Titova', 'MB Vrbanski', 'MS Cankarjeva', 'MS Rakičan',
       'NG Grčna', 'Novo mesto', 'Otlica', 'Ptuj', 'Rečica v I.Bistrici',
       'Trbovlje', 'Zagorje', 'promet'],
      dtype='object')

In [365]:
df.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [366]:
####

In [367]:
#PROMET - naloga 2

In [368]:
df_promet = pd.read_csv("data/raw/promet.csv", sep = ";", decimal = ".", encoding = "cp1250")
df_promet.head()

,Kat. ceste,Štev. ceste,Štev. odseka,Prometni odsek,Stac. začetka,Stac. konca,Števno mesto,Ime števnega mesta,Tip štetja,Vsa vozila (PLDP),Motorji,Osebna vozila,Avtobusi,"Lah. tov. < 3,5t","Sr. tov. 3,5-7t",Tež. tov. nad 7t,Tov. s prik.,Vlačilci,Dnevni NOO,Tip
0,AC,A1,30.0,MP ŠENTILJ - POČIVALIŠČE ŠENTILJ,0.0,482.000,674.0,MT Šentilj AC,QLTC8,18.686,34,12.884,47.0,2.030,226.0,126.0,407.0,2.932,2.816,PLDP
1,AC,A1,30.0,POČIVALIŠČE ŠENTILJ - ŠENTILJ,482.0,1.137,869.0,Šentilj AC,QLTC8,18.560,34,12.789,47.0,2.013,225.0,126.0,405.0,2.921,2.805,PLDP
2,AC,A1,199.0,PRIKLJ.ŠENTILJ S,0.0,842.000,868.0,Priklj. Šentilj S,QLTC8,284.000,1,239.000,0.0,26.000,2.0,1.0,2.0,13.000,13.000,PLDP
3,AC,A1,31.0,ŠENTILJ - PESNICA,0.0,9.761,837.0,Pesnica AC,QLTC8,25.324,49,18.784,80.0,2.707,204.0,94.0,485.0,2.921,2.875,PLDP
4,AC,A1,65.0,PESNICA - DRAGUČOVA,0.0,2.570,NaN,NaN,P,25.500,60,18.735,85.0,2.800,210.0,110.0,600.0,2.900,2.961,PLDP


In [369]:
#df_promet_filtered = df_promet[df_promet["Prometni odsek"].str.contains("CELJE|HRASTNIK|ISKRBA|KOPER|KRANJ|KRVAVEC|LJUBLJANA|BEŽIGRAD|CELOVŠKA|VIČ|MARIBOR|TITOVA|VRBANSKI|MURSKA SOBOTA|CANKARJEVA|RAKIČANNOVA GORICA|GRČNA|NOVO MESTO|OTILICA|PTUJ|REČICA|IL.|TRBOVLJE|ZAGORJE")==True]
df_promet_filtered = df_promet[df_promet["Prometni odsek"].str.contains("CELJE|HRASTNIK|ISKRBA|KOPER|KRANJ|KRVAVEC|LJUBLJANA|BEŽIGRAD|CELOVŠKA|VIČ|MARIBOR|TITOVA|VRBANSKI|MURSKA SOBOTA|CANKARJEVA|RAKIČAN|NOVA GORICA|GRČNA|NOVO MESTO|OTLICA|PTUJ|REČICA|IL\.|TRBOVLJE|ZAGORJE")==True]

df_promet_filtered

,Kat. ceste,Štev. ceste,Štev. odseka,Prometni odsek,Stac. začetka,Stac. konca,Števno mesto,Ime števnega mesta,Tip štetja,Vsa vozila (PLDP),Motorji,Osebna vozila,Avtobusi,"Lah. tov. < 3,5t","Sr. tov. 3,5-7t",Tež. tov. nad 7t,Tov. s prik.,Vlačilci,Dnevni NOO,Tip
6,AC,A1,71.0,MB (ZRKOVSKA) - MB (PTUJSKA),4.452,7.744,NaN,NaN,P,35.000,60,22.530,190.0,4.200,620.0,250.0,1.1,6.050,6.127,PLDP
7,AC,A1,66.0,MB (PTUJSKA) - ROGOZA,0.000,2.590,840.0,Ptujska AC,QLTC8,41.976,87,28.404,157.0,5.139,626.0,299.0,912.0,6.352,6.214,PLDP
24,AC,A1,39.0,LJUBEČNA - CELJE,6.436,8.438,1030.0,Celje Center-V AC,QLTC8,38.286,87,25.912,131.0,4.906,665.0,381.0,801.0,5.403,5.388,PLDP
25,AC,A1,40.0,CELJE - CELJE ZAHOD,0.000,5.261,648.0,Lopata AC,QLTC8,42.302,97,29.020,139.0,5.475,784.0,418.0,874.0,5.495,5.574,PLDP
26,AC,A1,40.0,CELJE ZAHOD - ARJA VAS,5.261,7.509,1031.0,Arja vas AC,QLTC8,47.450,120,33.578,156.0,5.694,886.0,483.0,892.0,5.641,5.770,PLDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,R3,713,4910.0,KRIŽ.SPUHLJA - PTUJ,28.500,31.230,425.0,Rogoznica,QLTC10,9.928,57,8.990,22.0,653.000,87.0,70.0,21.0,28.000,168.000,WIM
1102,R3,741,1487.0,DEKANI - PRIKLJ. LUKA KOPER,0.000,2.215,NaN,NaN,P,2.200,20,2.048,60.0,25.000,25.0,15.0,2.0,5.000,42.000,PLDP
1121,R3,751,5507.0,NIZKA - REČICA OB SAVINJI,0.000,905.000,NaN,NaN,P,1.700,10,1.310,50.0,150.000,60.0,50.0,30.0,40.000,102.000,PLDP
1152,RT,915,363.0,IL.BISTRICA (DO PODGRAJSKE UL.),0.000,182.000,NaN,NaN,P,8.500,110,7.785,75.0,400.000,65.0,20.0,20.0,25.000,83.000,PLDP


In [370]:
#df_promet_filtered = df_promet_filtered.drop(['Kat. ceste', 'Štev. ceste', 'Štev. odseka', 'Stac. začetka', 'Stac. konca',
#                                              'Števno mesto', 'Ime števnega mesta', 'Tip   štetja',
#                                             '', '', '', ''], axis=1)

In [371]:
df_promet_filtered = df_promet_filtered[['Prometni odsek', 'Vsa vozila (PLDP)']]

In [372]:
df_promet_filtered

,Prometni odsek,Vsa vozila (PLDP)
6,MB (ZRKOVSKA) - MB (PTUJSKA),35.000
7,MB (PTUJSKA) - ROGOZA,41.976
24,LJUBEČNA - CELJE,38.286
25,CELJE - CELJE ZAHOD,42.302
26,CELJE ZAHOD - ARJA VAS,47.450
...,...,...
1041,KRIŽ.SPUHLJA - PTUJ,9.928
1102,DEKANI - PRIKLJ. LUKA KOPER,2.200
1121,NIZKA - REČICA OB SAVINJI,1.700
1152,IL.BISTRICA (DO PODGRAJSKE UL.),8.500


In [373]:
df_promet_filtered.dtypes

Prometni odsek        object
Vsa vozila (PLDP)    float64
dtype: object

In [374]:
cat_cols = df_promet_filtered.select_dtypes(include=['object']).columns
print(cat_cols)
num_cols = df_promet_filtered.select_dtypes(include=['number']).columns
print(num_cols)

Index(['Prometni odsek'], dtype='object')
Index(['Vsa vozila (PLDP)'], dtype='object')


In [375]:
df_promet_celje = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("CELJE")==True]
df_promet_celje = df_promet_celje["Vsa vozila (PLDP)"].sum()
df_promet_celje

221.894

In [376]:
df_promet_hrastnik = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("HRASTNIK")==True]
df_promet_hrastnik = df_promet_hrastnik["Vsa vozila (PLDP)"].sum()
df_promet_hrastnik

285.955

In [377]:
df_promet_iskrba = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("ISKRBA")==True]
df_promet_iskrba = df_promet_iskrba["Vsa vozila (PLDP)"].sum()
df_promet_iskrba

0.0

In [378]:
df_promet_koper = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("KOPER")==True]
df_promet_koper = df_promet_koper["Vsa vozila (PLDP)"].sum()
df_promet_koper

113.777

In [379]:
df_promet_kranj = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("KRANJ")==True]
df_promet_kranj = df_promet_kranj["Vsa vozila (PLDP)"].sum()
df_promet_kranj

215.248

In [380]:
df_promet_krvavec = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("KRVAVEC")==True]
df_promet_krvavec = df_promet_krvavec["Vsa vozila (PLDP)"].sum()
df_promet_krvavec

0.0

In [381]:
df_promet_jubljana = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("LJUBLJANA|BEŽIGRAD|CELOVŠKA|VIČ")==True]
df_promet_jubljana = df_promet_jubljana["Vsa vozila (PLDP)"].sum()
df_promet_jubljana

1326.4769999999999

In [382]:
df_promet_maribor = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("MARIBOR|TITOVA|VRBANSKI")==True]
df_promet_maribor = df_promet_maribor["Vsa vozila (PLDP)"].sum()
df_promet_maribor

70.612

In [383]:
df_promet_murska_sobota = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("MURSKA SOBOTA|CANKARJEVA|RAKIČAN")==True]
df_promet_murska_sobota = df_promet_murska_sobota["Vsa vozila (PLDP)"].sum()
df_promet_murska_sobota

74.1

In [384]:
df_promet_nova_gorica = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("NOVA GORICA|GRČNA")==True]
df_promet_nova_gorica = df_promet_nova_gorica["Vsa vozila (PLDP)"].sum()
df_promet_nova_gorica

37.199

In [385]:
df_promet_novo_mesto = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("NOVO MESTO")==True]
df_promet_novo_mesto = df_promet_novo_mesto["Vsa vozila (PLDP)"].sum()
df_promet_novo_mesto

90.955

In [386]:
df_promet_otlica = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("OTLICA")==True]
df_promet_otlica = df_promet_otlica["Vsa vozila (PLDP)"].sum()
df_promet_otlica

0.0

In [387]:
df_promet_ptuj = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("PTUJ")==True]
df_promet_ptuj = df_promet_ptuj["Vsa vozila (PLDP)"].sum()
df_promet_ptuj

212.28699999999998

In [388]:
df_promet_il_bistrica = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("REČICA|IL\.")==True]
df_promet_il_bistrica = df_promet_il_bistrica["Vsa vozila (PLDP)"].sum()
df_promet_il_bistrica

1247.455

In [389]:
df_promet_trbovlje = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("TRBOVLJE")==True]
df_promet_trbovlje = df_promet_trbovlje["Vsa vozila (PLDP)"].sum()
df_promet_trbovlje

298.169

In [390]:
df_promet_zagorje = df_promet_filtered[df_promet_filtered["Prometni odsek"].str.contains("ZAGORJE")==True]
df_promet_zagorje = df_promet_zagorje["Vsa vozila (PLDP)"].sum()
df_promet_zagorje

32.744

In [391]:
df2 = df

In [395]:
columns = ['pm2.5', 'nadm_visina', 'o3', 'benzen', 'ge_sirina', 'pm10', 'co',
       'no2', 'ge_dolzina', 'so2', 'CE Ljubljanska', 'CE bolnica', 'Hrastnik',
       'Iskrba', 'Koper', 'Kranj', 'Krvavec', 'LJ Bežigrad', 'LJ Celovška',
       'LJ Vič', 'MB Titova', 'MB Vrbanski', 'MS Cankarjeva', 'MS Rakičan',
       'NG Grčna', 'Novo mesto', 'Otlica', 'Ptuj', 'Rečica v I.Bistrici',
       'Trbovlje', 'Zagorje']
df2 = df.reindex(columns=columns)

In [396]:
df.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [397]:
columns

['pm2.5',
 'nadm_visina',
 'o3',
 'benzen',
 'ge_sirina',
 'pm10',
 'co',
 'no2',
 'ge_dolzina',
 'so2',
 'CE Ljubljanska',
 'CE bolnica',
 'Hrastnik',
 'Iskrba',
 'Koper',
 'Kranj',
 'Krvavec',
 'LJ Bežigrad',
 'LJ Celovška',
 'LJ Vič',
 'MB Titova',
 'MB Vrbanski',
 'MS Cankarjeva',
 'MS Rakičan',
 'NG Grčna',
 'Novo mesto',
 'Otlica',
 'Ptuj',
 'Rečica v I.Bistrici',
 'Trbovlje',
 'Zagorje']

In [398]:
df2["promet"] = df2.apply(lambda row: df_promet_celje if row["CE Ljubljanska"] == 1
                          else df_promet_celje if row["CE bolnica"] == 1
                          else df_promet_hrastnik if row["Hrastnik"] == 1
                          else df_promet_iskrba if row["Iskrba"] == 1
                          else df_promet_koper if row["Koper"] == 1
                          else df_promet_kranj if row["Kranj"] == 1
                          else df_promet_krvavec if row["Krvavec"] == 1
                          else df_promet_jubljana if row["LJ Bežigrad"] == 1
                          else df_promet_jubljana if row["LJ Celovška"] == 1
                          else df_promet_jubljana if row["LJ Vič"] == 1
                          else df_promet_maribor if row["MB Titova"] == 1
                          else df_promet_maribor if row["MB Vrbanski"] == 1
                          else df_promet_murska_sobota if row["MS Cankarjeva"] == 1
                          else df_promet_murska_sobota if row["MS Rakičan"] == 1
                          else df_promet_nova_gorica if row["NG Grčna"] == 1
                          else df_promet_novo_mesto if row["Novo mesto"] == 1
                          else df_promet_otlica if row["Otlica"] == 1
                          else df_promet_ptuj if row["Ptuj"] == 1
                          else df_promet_il_bistrica if row["Rečica v I.Bistrici"] == 1
                          else df_promet_trbovlje if row["Trbovlje"] == 1
                          else df_promet_zagorje if row["Zagorje"] == 1
                          else None, axis=1)


In [399]:
df2

,pm2.5,nadm_visina,o3,benzen,ge_sirina,pm10,co,no2,ge_dolzina,so2,...,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje,promet
0,34.000000,299,61.000000,1.015454,46.065851,39.000000,0.300000,19.000000,14.517454,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
1,38.000000,305,54.802235,1.015454,46.072399,46.000000,0.401293,38.000000,14.491849,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
2,35.000000,293,54.802235,1.015454,46.037791,40.000000,0.401293,21.014164,14.494001,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
3,88.000000,388,54.802235,1.015454,46.242115,95.000000,0.401293,21.014164,14.366963,2.424805,...,0,0,0,0,0,0,0,0,0,215.248
4,33.000000,270,54.802235,2.300000,46.559202,41.000000,0.401293,32.000000,15.656191,2.424805,...,0,0,0,0,0,0,0,0,0,70.612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152035,26.000000,241,17.000000,1.015454,46.131416,30.000000,0.401293,30.000000,15.000906,4.000000,...,0,0,0,0,0,0,0,0,1,32.744
152036,14.000000,290,54.802235,1.015454,46.144213,15.000000,0.401293,21.014164,15.088097,2.424805,...,0,0,0,0,0,0,0,0,0,285.955
152037,41.000000,220,9.000000,1.015454,45.802080,44.000000,0.401293,19.000000,15.182060,2.424805,...,0,0,0,1,0,0,0,0,0,90.955
152038,10.000000,540,36.000000,1.015454,45.561344,13.000000,0.401293,4.000000,14.862816,2.000000,...,0,0,0,0,0,0,0,0,0,0.000


In [400]:
cat_cols_df2 = df2.select_dtypes(include=['object']).columns
print(cat_cols_df2)
num_cols_df2 = df2.select_dtypes(include=['number']).columns
print(num_cols_df2)

Index([], dtype='object')
Index(['pm2.5', 'nadm_visina', 'o3', 'benzen', 'ge_sirina', 'pm10', 'co',
       'no2', 'ge_dolzina', 'so2', 'CE Ljubljanska', 'CE bolnica', 'Hrastnik',
       'Iskrba', 'Koper', 'Kranj', 'Krvavec', 'LJ Bežigrad', 'LJ Celovška',
       'LJ Vič', 'MB Titova', 'MB Vrbanski', 'MS Cankarjeva', 'MS Rakičan',
       'NG Grčna', 'Novo mesto', 'Otlica', 'Ptuj', 'Rečica v I.Bistrici',
       'Trbovlje', 'Zagorje', 'promet'],
      dtype='object')


In [401]:
df2.dtypes

pm2.5                  float64
nadm_visina              int64
o3                     float64
benzen                 float64
ge_sirina              float64
pm10                   float64
co                     float64
no2                    float64
ge_dolzina             float64
so2                    float64
CE Ljubljanska           uint8
CE bolnica               uint8
Hrastnik                 uint8
Iskrba                   uint8
Koper                    uint8
Kranj                    uint8
Krvavec                  uint8
LJ Bežigrad              uint8
LJ Celovška              uint8
LJ Vič                   uint8
MB Titova                uint8
MB Vrbanski              uint8
MS Cankarjeva            uint8
MS Rakičan               uint8
NG Grčna                 uint8
Novo mesto               uint8
Otlica                   uint8
Ptuj                     uint8
Rečica v I.Bistrici      uint8
Trbovlje                 uint8
Zagorje                  uint8
promet                 float64
dtype: o

In [402]:
df2.isnull().sum()

pm2.5                  0
nadm_visina            0
o3                     0
benzen                 0
ge_sirina              0
pm10                   0
co                     0
no2                    0
ge_dolzina             0
so2                    0
CE Ljubljanska         0
CE bolnica             0
Hrastnik               0
Iskrba                 0
Koper                  0
Kranj                  0
Krvavec                0
LJ Bežigrad            0
LJ Celovška            0
LJ Vič                 0
MB Titova              0
MB Vrbanski            0
MS Cankarjeva          0
MS Rakičan             0
NG Grčna               0
Novo mesto             0
Otlica                 0
Ptuj                   0
Rečica v I.Bistrici    0
Trbovlje               0
Zagorje                0
promet                 0
dtype: int64

In [407]:
df2

,pm2.5,nadm_visina,o3,benzen,ge_sirina,pm10,co,no2,ge_dolzina,so2,...,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje,promet
0,34.000000,299,61.000000,1.015454,46.065851,39.000000,0.300000,19.000000,14.517454,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
1,38.000000,305,54.802235,1.015454,46.072399,46.000000,0.401293,38.000000,14.491849,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
2,35.000000,293,54.802235,1.015454,46.037791,40.000000,0.401293,21.014164,14.494001,2.424805,...,0,0,0,0,0,0,0,0,0,1326.477
3,88.000000,388,54.802235,1.015454,46.242115,95.000000,0.401293,21.014164,14.366963,2.424805,...,0,0,0,0,0,0,0,0,0,215.248
4,33.000000,270,54.802235,2.300000,46.559202,41.000000,0.401293,32.000000,15.656191,2.424805,...,0,0,0,0,0,0,0,0,0,70.612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152035,26.000000,241,17.000000,1.015454,46.131416,30.000000,0.401293,30.000000,15.000906,4.000000,...,0,0,0,0,0,0,0,0,1,32.744
152036,14.000000,290,54.802235,1.015454,46.144213,15.000000,0.401293,21.014164,15.088097,2.424805,...,0,0,0,0,0,0,0,0,0,285.955
152037,41.000000,220,9.000000,1.015454,45.802080,44.000000,0.401293,19.000000,15.182060,2.424805,...,0,0,0,1,0,0,0,0,0,90.955
152038,10.000000,540,36.000000,1.015454,45.561344,13.000000,0.401293,4.000000,14.862816,2.000000,...,0,0,0,0,0,0,0,0,0,0.000


In [403]:
df2.to_csv('../data/processed/data_traffic.csv', index=False)

In [404]:
df2.to_csv('data/processed/data_traffic.csv', index=False)

In [405]:
df2.to_csv('../src/data/data/processed/data_traffic.csv', index=False)